In [21]:
import pandas as pd
import numpy as np

profession_ds_sample = pd.read_csv(r"C:\Users\Admin\24[7]_Project\processed_text\profession_ds2.csv",encoding="utf-8",header=0, index_col=False)

list_of_rows = profession_ds_sample.to_numpy().tolist()

sentences = []
classes = []

print("Initializing categories...")
categories = ["administrator","assistant","banker","doctor","military personnel","musician","painter","photographer","policeman","priest","professor","soldier","sportsman","student"]

def class_label(lines_cls):
    return_list = []
    ind = 0
    for items in categories:
        if lines_cls == items:
            return_list.append(1)
        else:
            return_list.append(0)
    return return_list

def isNaN(string):
    return string != string


def max_ind_fn(output_list):
    ind = 0
    max_ind = 0
    key = output_list[0]
    for item in output_list:
        if(key<item):
            key = item
            max_ind = ind
        ind+=1
    return max_ind

print("Loading dialogues and classes...")
for lines in list_of_rows:
    sentences.append(lines[1])
    classes.append(class_label(lines[0]))

print("Loading dialogues and classes completed...")


Initializing categories...
Loading dialogues and classes...
Loading dialogues and classes completed...


In [22]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

print("Loading stopwords for removal...")
stop_words = set(stopwords.words('english')) 


def remove_punct(strin):
    punct_str = '!"#$%&\'()*+,-./:;?@[\]^_`{|}~'
    restr = ""
    for letter in strin:
        if(letter not in punct_str):
            restr = restr+letter
    return restr

Loading stopwords for removal...


In [23]:
docs = []
labels_y = []
max_length_in_words = 0

ind = 0 
for lines in sentences:
    if(not isNaN(lines)):
        try:
            clean_sent = remove_punct(lines).strip()
            clean_words = clean_sent.split()
            pure_sentence = ""
            for words in clean_words:
                if(len(words)>1 and words.lower() not in stop_words):
                    pure_sentence += words.lower()+" "
            if(len(pure_sentence.split())>max_length_in_words):
                max_length_in_words = len(pure_sentence)
            docs.append(pure_sentence.strip())
            labels_y.append(classes[ind])
        except:
            print(lines)
        ind+=1
#docs

In [27]:
print("Importing tensorflow libraries...")
import tensorflow as tffrom 
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten, Conv1D
from tensorflow.keras.layers import Embedding

classes = labels_y
#labels = np.array(class_labels)
labels = np.array(classes)
vocab_size = 2000

print("Encoding given words to values...")
print("Max length in words - ", max_length_in_words)

encoded_docs = [one_hot(d, vocab_size) for d in docs]
#print(encoded_docs)
max_length = max_length_in_words
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)
print("Defining the CNN model. Summary is provided below")
model = Sequential()
model.add(Embedding(vocab_size, 14, input_length=max_length))
model.add(Dropout(0.2))
model.add(Conv1D(filters=14,kernel_size=3,padding='valid', activation='relu', strides=1))
model.add(Flatten())
model.add(Dense(14, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
print("Fitting the model with data...")
model.fit(padded_docs, labels, epochs=100, verbose=1)
# evaluate the model
print("Calculating accuracy...")
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Importing tensorflow libraries...
Encoding given words to values...
Max length in words -  204
Defining the CNN model. Summary is provided below
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 204, 14)           28000     
_________________________________________________________________
dropout_3 (Dropout)          (None, 204, 14)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 202, 14)           602       
_________________________________________________________________
flatten_3 (Flatten)          (None, 2828)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 14)                39606     
Total params: 68,208
Trainable params: 68,208
Non-trainable params: 0
_____________________________________

3322/3322 [==============================] - 1s 416us/sample - loss: 0.0715 - accuracy: 0.9748
Epoch 65/100
3322/3322 [==============================] - 1s 390us/sample - loss: 0.0698 - accuracy: 0.9759
Epoch 66/100
3322/3322 [==============================] - 1s 340us/sample - loss: 0.0710 - accuracy: 0.9748
Epoch 67/100
3322/3322 [==============================] - 1s 319us/sample - loss: 0.0703 - accuracy: 0.9750
Epoch 68/100
3322/3322 [==============================] - 1s 317us/sample - loss: 0.0700 - accuracy: 0.9753
Epoch 69/100
3322/3322 [==============================] - 1s 313us/sample - loss: 0.0702 - accuracy: 0.9753
Epoch 70/100
3322/3322 [==============================] - 1s 341us/sample - loss: 0.0678 - accuracy: 0.9761
Epoch 71/100
3322/3322 [==============================] - 1s 334us/sample - loss: 0.0677 - accuracy: 0.9758
Epoch 72/100
3322/3322 [==============================] - 1s 344us/sample - loss: 0.0686 - accuracy: 0.9755
Epoch 73/100
3322/3322 [=================

In [30]:
model.save("profession_cnn_model1.model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: profession_cnn_model1.model\assets


In [40]:
result = model.predict(padded_docs[2500:2501])

for res_item in result:
    print(categories[max_ind_fn(res_item)])

military personnel
